In [1]:
#from mlxtend.frequent_patterns import apriori
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from scipy.io import arff
data, meta = arff.loadarff('./data/supermarket.arff')

data

array([(b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b't', b't', b't', b't', b'?', b't', b'?', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b't', b't', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b't', b'?', b'?', b't', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b't', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'?', b'

In [3]:
supermarket_one_hot = pd.get_dummies(pd.DataFrame(data))
# find cols with interrogation marks
cols_with_interrogation_mark = supermarket_one_hot.filter(regex='\'\?\'$',axis=1).columns # Fix regex as it did not match any cols on my environment...
# delete columsn with interrogation marks in it
supermarket_one_hot.drop(cols_with_interrogation_mark,axis=1,inplace=True)

In [11]:
from mlxtend.frequent_patterns import apriori
apriori_result = apriori(supermarket_one_hot, min_support=0.1)
print('apriori_result n of rows : {}'.format(apriori_result.shape[0]))

apriori_result n of rows : 10282
        support                       itemsets
0      0.226281                            (0)
1      0.133780                           (10)
2      0.719689                           (11)
3      0.604063                           (12)
4      0.532310                           (13)
...         ...                            ...
10277  0.101145  (11, 77, 15, 80, 24, 122, 29)
10278  0.100713   (37, 11, 77, 15, 80, 57, 29)
10279  0.103307  (37, 11, 77, 15, 80, 122, 29)
10280  0.102658  (38, 11, 77, 15, 80, 122, 29)
10281  0.105036  (11, 77, 15, 80, 57, 122, 29)

[10282 rows x 2 columns]


In [5]:
# generate a bar chart, itemsets frequencies in terms of items containes
import matplotlib
import matplotlib.pyplot as plt

max_itemsets_len = apriori_result.itemsets.map(len).max()
frequencies = {}

for i in range(1, max_itemsets_len + 1):

x = np.arange(len(frequencies.keys()))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()

rects = ax.bar(x - width/2, frequencies.values(), width, label='itemsets')

# Add value to each bar
for rect in rects:
    height = rect.get_height()
    ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequency')
ax.set_xlabel('Items contained')
ax.set_title('itemsets frequencies')
ax.set_xticks(x)
ax.set_xticklabels(frequencies.keys())
ax.legend()
plt.savefig('fig/itemsets_frequencies')
plt.close(fig)

In [6]:
# build association rules
from mlxtend.frequent_patterns import association_rules
min_threshold = 0.7
dataset_association_rules = association_rules(apriori_result, min_threshold=min_threshold)
print('Number of association rules : {}'.format(dataset_association_rules.shape[0]))
print('-------------')
print('First association rule : ')
print(dataset_association_rules.iloc[0])

Number of association rules : 24570
-------------
First association rule : 
antecedents                  (0)
consequents                 (11)
antecedent support      0.226281
consequent support      0.719689
support                 0.171601
confidence              0.758357
lift                     1.05373
leverage              0.00874991
conviction               1.16002
Name: 0, dtype: object


In [8]:
dataset_association_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(0),(11),0.226281,0.719689,0.171601,0.758357,1.053729,0.008750,1.160023
1,(10),(11),0.133780,0.719689,0.100929,0.754443,1.048290,0.004649,1.141531
2,(12),(11),0.604063,0.719689,0.473525,0.783900,1.089221,0.038788,1.297136
3,(13),(11),0.532310,0.719689,0.403933,0.758831,1.054387,0.020836,1.162301
4,(14),(11),0.193646,0.719689,0.153231,0.791295,1.099496,0.013866,1.343095
